In [1]:
from openai import OpenAI
import configparser
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:

model_word = KeyedVectors.load_word2vec_format('../../Modell_Fast_text/wiki-news-300d-1M-subword.vec', binary=False)

In [56]:
final_table = pd.DataFrame(columns=["Question", "Answer", "Answer1", "Answer2", "Result", "similarity_1", "similarity_2","similarity_winner"])

In [3]:
def text_to_vector(text, model):
    words = text.split()
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)


def cosine_similarity_texts(text1, text2, model):
    vector1 = text_to_vector(text1, model)
    vector2 = text_to_vector(text2, model)
    return cosine_similarity([vector1], [vector2])[0][0]

In [4]:
config = configparser.ConfigParser()
config.read('../../Edu_Llama/config.ini')
api_key = config['openai']['api_key']

In [5]:
client = OpenAI(api_key=api_key)

In [6]:
llm1 = "gpt-3.5-turbo"
llm2 = "gpt-4-1106-preview"

In [7]:
questions = pd.read_csv("../../Fragen/Fragen DB.CSV", sep=";") 

In [8]:
questions.head()


,Question,Answer
0,Question: What are the basic features of Relat...,Answer: The basic features of RDMS include the...
1,Question: What are the key components of a dat...,Answer: Key components of a database system en...
2,Question: What is a relation (relation state) ...,Answer: A relation (relation state) in relatio...
3,Question: What is the definition and significa...,Answer: A binary relation in relational databa...
4,Question: What are entity types and sets in th...,Answer: An entity type is a definition of a co...


In [9]:
question = questions["Question"][0]
answer = questions["Answer"][0]

In [10]:
response1 = client.chat.completions.create(
    model=llm1,
    messages=[{"role": "user", "content": f"{question}"}]
)
answer1 = response1.choices[0].message.content

In [11]:
response2 = client.chat.completions.create(
    model=llm2,
    messages=[{"role": "user", "content": f"{question}"}]
)
answer2 = response2.choices[0].message.content

In [66]:
import random

zufallszahl = random.choice([0, 1])
answers= [answer1, answer2]

if zufallszahl == 0:
    zufallszahl2 = 1
else:
    zufallszahl2 = 0

In [15]:
auswertung = client.chat.completions.create(
    model=llm2,
    messages=[{"role": "system", "content": "I'll now give you a question, the sample solution to this question and 2 other answers. Decide based on the context of the model solution which of the two answers is better and only output “1” or “2” as the result!"},
              {"role": "user", "content": f"{question} \n sample {answer} \n 1. Answer: {answers[zufallszahl]} \n 2. Answer: {answers[zufallszahl2]}"}]
)

In [68]:
better_answer_llm = auswertung.choices[0].message.content
better_answer_llm = int(better_answer_llm)

In [69]:
better_answer_llm

1

In [70]:
f"{question} \n sample {answer} \n 1. Answer: {answer1} \n 2. Answer: {answer2}"

"Question: What are the basic features of Relational Database Management Systems (RDMS)? \n sample Answer: The basic features of RDMS include the ability to create a database, write and execute SQL queries, evaluate storage structures, read execution plans and optimize queries, and understand transaction isolation layers. \n 1. Answer: Relational Database Management Systems (RDBMS) have several basic features that differentiate them from other types of database management systems. Here are some of the key features:\n\n1. Structure: RDBMS organizes data into tables with rows and columns, where each table represents a specific entity (e.g., customers, products). The columns represent attributes or characteristics of the entity, and the rows represent individual records or instances.\n\n2. Data Integrity: RDBMS enforce data integrity constraints to maintain the accuracy and consistency of data. These constraints include primary key constraints, foreign key constraints, unique constraints,

### Kosinusähnlichkeit

In [71]:
cos_1 = cosine_similarity_texts(answer, answer1, model_word)
cos_2 = cosine_similarity_texts(answer, answer2, model_word)

In [72]:
if cos_1 > cos_2:
    better_answer = 1
else:
    better_answer = 2

### Final Data Frame

In [73]:
if zufallszahl == 1:
    if int(better_answer_llm) == 1:
        better_answer_llm = 2
    else:
        better_answer_llm = 1

In [74]:
better_answer_llm

2

In [ ]:
# Einkommentieren falls tabelle gelöscht werden soll:
# final_table = pd.DataFrame(columns=["Question", "Answer", "Answer1", "Answer2", "Result", "similarity_1", "similarity_2","similarity_winner"])

In [75]:
neue_zeile = [question, answer, answer1, answer2, better_answer_llm,cos_1, cos_2,better_answer]

final_table.loc[len(final_table)] = neue_zeile

In [76]:
final_table

,Question,Answer,Answer1,Answer2,Result,similarity_1,similarity_2,similarity_winner
0,Question: What are the basic features of Relat...,Answer: The basic features of RDMS include the...,Relational Database Management Systems (RDBMS)...,Relational Database Management Systems (RDBMS)...,1,0.953682,0.957492,2
1,Question: What are the basic features of Relat...,Answer: The basic features of RDMS include the...,Relational Database Management Systems (RDBMS)...,Relational Database Management Systems (RDBMS)...,1,0.953682,0.957492,2
2,Question: What are the basic features of Relat...,Answer: The basic features of RDMS include the...,Relational Database Management Systems (RDBMS)...,Relational Database Management Systems (RDBMS)...,2,0.953682,0.957492,2


In [61]:
file_name = 'Eval.csv'
final_table.to_csv(file_name, index=False)